In [1]:
import sys
sys.path.append("../")

In [2]:
from loss import compute_loss
import networks as net
from data_sampler import data_sampler

import functools
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

In [3]:
train_pull = pd.read_csv("../../../data/fashion_mnisit/train_pull.csv", header=None).values
train_top = pd.read_csv("../../../data/fashion_mnisit/train_top.csv", header=None).values
# test_pull = pd.read_csv("../../data/fashion_mnisit/test_pull.csv", header=None).values
# test_top = pd.read_csv("../../data/fashion_mnisit/test_top.csv", header=None).values

In [4]:
input_nc = 1
output_nc = 1
discr_filters = 8
max_power = 8
n_layers = 3
norm_lay = nn.BatchNorm2d
start_size = 28
gen_filters = 16
dropout = None
n_blocks = 1

In [5]:
discr_a = net.Discrimanator(input_nc=input_nc,
                  discr_filters=discr_filters,
                  max_power=max_power,
                  n_layers=n_layers,
                  norm_lay=norm_lay,
                  start_size=28)

discr_b = net.Discrimanator(input_nc=input_nc,
                  discr_filters=discr_filters,
                  max_power=max_power,
                  n_layers=n_layers,
                  norm_lay=norm_lay,
                  start_size=28)

gener_a = net.ResnetGenerator(
    input_nc = input_nc,
    output_nc = output_nc,
    gen_filters = gen_filters,
    norm_lay = norm_lay,
    dropout = dropout,
    n_blocks = n_blocks
)

gener_b = net.ResnetGenerator(
    input_nc = input_nc,
    output_nc = output_nc,
    gen_filters = gen_filters,
    norm_lay = norm_lay,
    dropout = dropout,
    n_blocks = n_blocks
)

In [15]:
batch_a, batch_b = data_sampler(10, train_pull, train_top)
batch_a = batch_a.view(-1, 1, 28, 28).float()
batch_b = batch_b.view(-1, 1, 28, 28).float()

In [16]:
compute_loss(
    gener_a = gener_a,
    gener_b = gener_b,
    discr_a = discr_a,
    discr_b = discr_b,
    batch_a = batch_a,
    batch_b = batch_b,
    alpha = 10
)

RuntimeError: input and target have different number of elements: input[10 x 1] has 10 elements, while target[1] has 1 elements at /Users/soumith/minicondabuild3/conda-bld/pytorch_1512381214802/work/torch/lib/THNN/generic/MSECriterion.c:13

In [17]:
discr_a_onrly = discr_a(batch_a)
discr_b_onrly = discr_b(batch_b)

fake_a = gener_a(batch_b)
fake_b = gener_b(batch_a)

discr_a_onfke = discr_a(fake_a)
discr_b_onfke = discr_b(fake_b)

cyclic_a = gener_a(fake_b)
cyclic_b = gener_a(fake_a)